#Artificial Neural Network (ANN)

In [29]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [30]:
tf.__version__

'2.12.0'

##Data Pre Processing

In [31]:
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:-1].values
y = dataset.iloc[:, -1].values

In [32]:
print(X)
print(X[:,2:3])

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]
[['Female']
 ['Female']
 ['Female']
 ...
 ['Female']
 ['Male']
 ['Female']]


In [33]:
print(y)
print(y.reshape((len(y), 1)))

[1 0 1 ... 1 1 0]
[[1]
 [0]
 [1]
 ...
 [1]
 [1]
 [0]]


##Encoding Categorical data

In [34]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:,2]=le.fit_transform(X[:,2])
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


##One Hot Encoding

In [35]:
print(X[0,:])

[619 'France' 0 42 2 0.0 1 1 1 101348.88]


In [36]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder',OneHotEncoder(), [1])], remainder='passthrough')
X=np.array(ct.fit_transform(X))

print(X)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


In [37]:
print(X[0,:]) # the one hot encoded values are moved to the begining of the array

[1.0 0.0 0.0 619 0 42 2 0.0 1 1 1 101348.88]


##Split Dataset into test set and training set

In [38]:
# this needs to be done before feature scaling as the model should not have any information about the test data (to avoid information leakage)
from sklearn.model_selection import train_test_split
X_train_unscaled, X_test_unscaled, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)


##Feature scaling

In [39]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train_unscaled) # We are scaling all the  feautures instead scaling only a few features
X_test = sc.transform(X_test_unscaled)

In [40]:
print("X_train: ", X_train)
print("X_test: ", X_test)

X_train:  [[-1.01460667 -0.5698444   1.74309049 ...  0.64259497 -1.03227043
   1.10643166]
 [-1.01460667  1.75486502 -0.57369368 ...  0.64259497  0.9687384
  -0.74866447]
 [ 0.98560362 -0.5698444  -0.57369368 ...  0.64259497 -1.03227043
   1.48533467]
 ...
 [ 0.98560362 -0.5698444  -0.57369368 ...  0.64259497 -1.03227043
   1.41231994]
 [-1.01460667 -0.5698444   1.74309049 ...  0.64259497  0.9687384
   0.84432121]
 [-1.01460667  1.75486502 -0.57369368 ...  0.64259497 -1.03227043
   0.32472465]]
X_test:  [[-1.01460667  1.75486502 -0.57369368 ...  0.64259497  0.9687384
   1.61085707]
 [ 0.98560362 -0.5698444  -0.57369368 ...  0.64259497 -1.03227043
   0.49587037]
 [-1.01460667 -0.5698444   1.74309049 ...  0.64259497  0.9687384
  -0.42478674]
 ...
 [-1.01460667 -0.5698444   1.74309049 ...  0.64259497 -1.03227043
   0.71888467]
 [-1.01460667  1.75486502 -0.57369368 ...  0.64259497  0.9687384
  -1.54507805]
 [-1.01460667  1.75486502 -0.57369368 ...  0.64259497 -1.03227043
   1.61255917]]


##Building the ANN

###Initialize ANN

In [41]:
ann=tf.keras.models.Sequential()
#add input layer and first hidden layer
##the number of neurons in the input layer is determined by the number of inputs
##the units parameter specifies the number of neurons in hidden layers. its value is arrived at based on experimentation with different values
##activation parameter determines the activation function used used in the neurons. 'relu' stands for rectifier activation function
ann.add(tf.keras.layers.Dense(units=6,activation='relu')) # Adding the first hidden layer (shallow NN)
#add output layer
ann.add(tf.keras.layers.Dense(units=1,activation='sigmoid')) # Adding the output layer with one binary output



##Train ANN

###compile ANN

In [42]:
ann.compile(optimizer= 'adam', loss='binary_crossentropy', metrics=['accuracy'])

###train

In [43]:
ann.fit(X_train, y_train, batch_size=32, epochs=100)

Epoch 1/100
250/250 [==============================] - 1s 2ms/step - loss: 0.5640 - accuracy: 0.7460
Epoch 2/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4698 - accuracy: 0.8019
Epoch 3/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4343 - accuracy: 0.8114
Epoch 4/100
250/250 [==============================] - 1s 4ms/step - loss: 0.4181 - accuracy: 0.8199
Epoch 5/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4080 - accuracy: 0.8280
Epoch 6/100
250/250 [==============================] - 1s 2ms/step - loss: 0.3996 - accuracy: 0.8314
Epoch 7/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3913 - accuracy: 0.8369
Epoch 8/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3838 - accuracy: 0.8424
Epoch 9/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3775 - accuracy: 0.8461
Epoch 10/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3726 - accura

##Predict using ANN

In [44]:
y_predict=ann.predict(X_test)

63/63 [==============================] - 0s 1ms/step


In [60]:
print(y_predict)
print(y_predict>0.5)
print(y_test)


[[0.27918148]
 [0.37258315]
 [0.17701256]
 ...
 [0.18597515]
 [0.19597086]
 [0.22209051]]
[[False]
 [False]
 [False]
 ...
 [False]
 [False]
 [False]]
[0 1 0 ... 0 0 0]


In [90]:
#Make confusion matrix
y_predict_=np.where(y_predict >0.5, 1,0)

print(y_predict_.T[0].shape)

y_test_=np.array(np.array(y_test))
print(y_test_.shape)

from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_predict_.T[0])
print(cm)
accuracy_score(y_test, y_predict_.T[0])


(2000,)
(2000,)
[[1510   85]
 [ 199  206]]


0.858

In [45]:
# 600,France,Male,40,3,60000,2,1,1,50000
d1=pd.read_csv('single.csv')
print(d1)


   CreditScore Geography Gender  Age  Tenure  Balance  NumOfProducts  \
0          600    France   Male   40       3    60000              2   

   HasCrCard  IsActiveMember  EstimatedSalary  
0          1               1            50000  


In [46]:
print(type(d1.values))
X1=d1.values
print(X1)
X1[:, 2] = le.transform(X1[:, 2])
print(X1)

<class 'numpy.ndarray'>
[[600 'France' 'Male' 40 3 60000 2 1 1 50000]]
[[600 'France' 1 40 3 60000 2 1 1 50000]]


In [47]:
X1 = np.array(ct.transform(X1))
print(X1)

[[1.0 0.0 0.0 600 1 40 3 60000 2 1 1 50000]]


In [51]:
X1Man=[[1,0,0,600,1,40,3,60000,2,1,1,50000]]
X1_test = sc.transform(X1)
X1man_test=sc.transform(X1Man)
print(X1_test)
print(X1man_test)

[[ 0.98560362 -0.5698444  -0.57369368 -0.52111599  0.91601335  0.10961719
  -0.68538967 -0.2569057   0.8095029   0.64259497  0.9687384  -0.87203322]]
[[ 0.98560362 -0.5698444  -0.57369368 -0.52111599  0.91601335  0.10961719
  -0.68538967 -0.2569057   0.8095029   0.64259497  0.9687384  -0.87203322]]


In [54]:
print(ann.predict(X1_test))
print(ann.predict(X1man_test))
print(ann.predict(X1_test)>0.5)
print(ann.predict(X1man_test)>0.5)

1/1 [==============================] - 0s 65ms/step
[[0.02898446]]
1/1 [==============================] - 0s 54ms/step
[[0.02898446]]
1/1 [==============================] - 0s 45ms/step
[[False]]
1/1 [==============================] - 0s 34ms/step
[[False]]
